In [38]:
#Lets start by importing the necessary libraries
import sqlalchemy as sqla
import pymysql
import pandas as pd
import os

username = "user"  # get this from the instructor
password = "grad5100user"  # get this from the instructor
awsresource = "database-1.cwvjklnp4wu3.us-east-1.rds.amazonaws.com"  # get this from the instructor
port = 3306

In [39]:
#Now lets get the database connection
dbname = "topChef"
dburi = f"mysql+pymysql://{username}:{password}@{awsresource}:{port}/{dbname}"
engine = sqla.create_engine(dburi)

In [40]:
#Now we will create two functions that will help us run SQL commands
def run_sql(sql, engine):
    "send sql to database given by engine and return the result"
    with engine.connect() as conn:
        result = conn.execute(sqla.text(sql))
    return result.all()


def df_select(sql, engine):
    "send sql to database given by engine and return the result as a pandas dataframe"
    with engine.connect() as conn:
        result = pd.read_sql(sqla.text(sql), con=conn)
    print(f"Retrieved {result.shape[0]} records")
    return result

In [41]:
#We will create a new table called chefdetails2 that will exclude the Masters and Canada seasons
with engine.connect() as conn:
    chefdetails2 = pd.read_sql(
        sqla.text("SELECT * FROM chefdetails WHERE season NOT LIKE '%Masters%' AND season NOT LIKE '%Canada%';"), conn
    )

#We will create a new table called judges2 that will exclude the Masters and Canada seasons
with engine.connect() as conn:
    judges2 = pd.read_sql(
        sqla.text("SELECT * FROM judges WHERE season NOT LIKE '%Masters%' AND season NOT LIKE '%Canada%';"), conn
    )

In [42]:
#Now we will only keep the columns that we are interested in from the chefdetails2 and judges2 tables
chefdetails2 = chefdetails2[['season','seasonNumber','name','placement','gender']]
judges2 = judges2[['season','episode','challengeType','guestJudge','competedOnTC']]

In [43]:
#Make a table that has two columns, one called season and one called seasonNumber showing the number associated with each season
seasonNumber = chefdetails2[['season','seasonNumber']]
seasonNumber = seasonNumber.drop_duplicates()
seasonNumber = seasonNumber.sort_values(by=['seasonNumber'])
seasonNumber = seasonNumber.reset_index(drop=True)
print(seasonNumber)

                 season  seasonNumber
0         San Francisco             1
1           Los Angeles             2
2                 Miami             3
3               Chicago             4
4              New York             5
5             Las Vegas             6
6                  D.C.             7
7   All Stars: New York             8
8                 Texas             9
9               Seattle            10
10          New Orleans            11
11               Boston            12
12           California            13
13           Charleston            14
14             Colorado            15
15             Kentucky            16
16       All-Stars L.A.            17
17             Portland            18
18              Houston            19
19      World All Stars            20


In [44]:
#Among all chef contestants, how many are male and how many are female?
Male, Female = chefdetails2.gender.value_counts()
print('There are ' + str(Male) + ' contestants who were male and ' + str(Female) + ' contestants who were female.')

There are 170 contestants who were male and 152 contestants who were female.


In [45]:
#Among all winners (placement==1) how many are male and how many are female?
Male, Female = chefdetails2[chefdetails2['placement']==1].gender.value_counts()
print('Among the contestants who won, there were ' + str(Male) + ' males and ' + str(Female) + ' females.')

Among the contestants who won, there were 14 males and 6 females.


In [46]:
#Among all sets of top 3 finalists (placement=1,2,3) how many are male and how many are female?
Male, Female = chefdetails2[chefdetails2['placement']<=3].gender.value_counts()
print('Among all sets of top 3 finalists, there were ' + str(Male) + ' males and ' + str(Female) + ' females.')


Among all sets of top 3 finalists, there were 36 males and 25 females.


Looking at the value counts of males and females through different stages of the competition, we do see some unusual behavior. Specifically, about 41% of top 3 finalists are women (25/(25+36)). However, when we determine the percentage of women who won the competetion, that number drops to only 30% (6/(6+14)). Therefore, women who place in the top 3 are less likely than expected to win the competition. 

In [47]:
#Now lets make a dataframe that contains the name of each winner, the season they won, and whether or not they were a guest judge
firstPlace = chefdetails2[chefdetails2['placement']==1]
firstPlace = firstPlace[['name','season']]
firstPlace = firstPlace.reset_index(drop=True)
if firstPlace['name'][0] in judges2['guestJudge'].values:
    firstPlace['guestJudge'] = 'Yes'
else:
    firstPlace['guestJudge'] = 'No'

print(firstPlace)

                    name               season guestJudge
0          Richard Blais  All Stars: New York        Yes
1           Melissa King       All-Stars L.A.        Yes
2                Mei Lin               Boston        Yes
3            Jeremy Ford           California        Yes
4      Brooke Williamson           Charleston        Yes
5        Stephanie Izard              Chicago        Yes
6           Kevin Sbraga                 D.C.        Yes
7           Joseph Flamm             Colorado        Yes
8             Kah-wai Lo              Houston        Yes
9   Kelsey Barnard Clark             Kentucky        Yes
10     Michael Voltaggio            Las Vegas        Yes
11             Ilan Hall          Los Angeles        Yes
12            Hung Huynh                Miami        Yes
13         Nicholas Elmi          New Orleans        Yes
14       Hosea Rosenberg             New York        Yes
15           Gabe Erales             Portland        Yes
16       Harold Dieterle       

In [54]:
# Find all contestants who were not first place winners but did serve as a guest judge at some point. Make sure to only list each name once.
notfirstPlace = chefdetails2[chefdetails2['placement']!=1]
notfirstPlace = notfirstPlace[['name']]
guestJudge = pd.DataFrame({'name': []})

for name in notfirstPlace['name']:
    if name in judges2['guestJudge'].values:
        guestJudge = pd.concat([guestJudge, pd.DataFrame({'name': [name]})], ignore_index=True)

guestJudge = guestJudge.drop_duplicates()
guestJudge = guestJudge.reset_index(drop=True)
print(guestJudge)



                  name
0        Mike Isabella
1       Antonia Lofaso
2        Tiffany Derry
3           Dale Talde
4     Spike Mendelsohn
5     Stephen Asprinio
6      Bryan Voltaggio
7      Gregory Gourdet
8      Karen Akunowicz
9        Lee Anne Wong
10         Nini Nguyen
11        Melissa King
12        Amar Santana
13      Kwame Onwuachi
14      Sheldon Simeon
15          Sam Talbot
16       Richard Blais
17   Adrienne Cheatham
18        Carrie Baird
19  Josie Smith-Malave
20        Nina Compton
21      Stefan Richter
22        Dawn Burrell
23      Shota Nakajima
24      Miguel Morales
25   Brooke Williamson
26          Edward Lee
27    Nyesha Arrington
